In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# page and congress
url_template="http://www.gpo.gov/fdsys/search/search.action?sr={0}&originalSearch=collection%3aCHRG&st=collection%3aCHRG&ps=100&na=__congressnum&se=__{1}true&sb=dno&timeFrame=&dateBrowse=&govAuthBrowse=&collection=&historical=true"

In [25]:

import re
# Session will work better to store connection state. Cookies!
s = requests.Session()
#load base page and setup session
s.get("https://www.gpo.gov/fdsys/")

def get_chrg_ids(page=1,congress=99):
    r=s.get(url_template.format(page,congress))
    soup=BeautifulSoup(r.text,'html.parser')
    links=[]
    for link in soup.findAll('a'):
        if link.get('href'):
            links.append(link.get('href'))
    valid_ids=[]
    for link in links:
        end_url=link.split('/')[-1] 
        if re.match('^CHRG*',end_url):
            valid_ids.append(end_url.split('.p')[0])
    return valid_ids

In [40]:

def get_ids(congress):
    cum_ids=[]
    for itm in range(1,1000):
        ids = get_chrg_ids(page=itm,congress=congress)
        if ids==[]:
            break
        cum_ids = cum_ids + ids
    return cum_ids  

In [33]:
total_ids=[]
for cong in range(99,115):
    total_ids =total_ids + get_ids(cong)

In [34]:
len(total_ids)
#check web search page for total number 24197
# Shit still missing 5 hearings! This is ok for now, but will need to 
# find the 5 missing ids. We can write a verify script down the road!
# Also can just pull first page to check for new hearings. 

24192

In [35]:
total_ids[:10]

['CHRG-99shrg63488',
 'CHRG-100shrg83712',
 'CHRG-102hhrg67539',
 'CHRG-103hhrg91083',
 'CHRG-103hhrg91082',
 'CHRG-103hhrg88390',
 'CHRG-103hhrg88392',
 'CHRG-103hhrg88391',
 'CHRG-103hhrg83241',
 'CHRG-103hhrg66111']

In [37]:
#This is your code!!!
from xmltodict import parse
import json
def modsParser(tag,url):
    xmlURL = url
    r = s.get(xmlURL)
    data = parse(r.text)
    data["tag"]=tag
    x = json.loads(json.dumps(data).replace("@",'').replace("#",''))
    print(x)
    #db = MongoClient("dsl_search_mongo",27017)
    #db.congressional.hearings.save(x)

In [39]:
modsURL_template =  "https://www.gpo.gov/fdsys/pkg/{0}/mods.xml"

#test the first 10 items
for chrg in total_ids[:10]:
    modsParser(chrg,modsURL_template.format(chrg))

{'tag': 'CHRG-99shrg63488', 'mods': {'xmlns:xlink': 'http://www.w3.org/1999/xlink', 'name': [{'role': [{'roleTerm': [{'text': 'printer', 'type': 'text', 'authority': 'marcrelator'}, {'text': 'prt', 'type': 'code', 'authority': 'marcrelator'}]}, {'roleTerm': [{'text': 'distributor', 'type': 'text', 'authority': 'marcrelator'}, {'text': 'dst', 'type': 'code', 'authority': 'marcrelator'}]}], 'namePart': 'United States Government Printing Office', 'type': 'corporate'}, {'role': {'roleTerm': [{'text': 'author', 'type': 'text', 'authority': 'marcrelator'}, {'text': 'aut', 'type': 'code', 'authority': 'marcrelator'}]}, 'namePart': ['United States', 'Congress', 'Senate', 'Select Committee on Indian Affairs'], 'type': 'corporate', 'description': 'Government Organization'}, {'role': {'roleTerm': [{'text': 'associated name', 'type': 'text', 'authority': 'marcrelator'}, {'text': 'asn', 'type': 'code', 'authority': 'marcrelator'}]}, 'namePart': ['United States', 'Congress', 'Senate', 'Select Commit